In [ ]:
default_cnt = 100000
default_mul = 1000
default_model_name = "model-" + str(default_cnt // 1000) + "k-mul" + str(default_mul) + "-all-bs60-8.bin"
default_wandb_name = "testing: " + default_model_name
print('name:', default_wandb_name)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from tqdm import tqdm
from pathlib import Path

import time

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

def is_interactive_mode():
    return os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Interactive') == 'Interactive'

data_dir = Path('../input/AI4Code')

df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')

cnt_by_group = {}
for id, row in tqdm(df_ancestors.iterrows()):
    cnt_by_group[row['ancestor_id']] = cnt_by_group.get(row['ancestor_id'], 0) + 1


cnt = pd.Series(cnt_by_group)
print('only one:', cnt[cnt == 1].count())
cnt.plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
cnt

good_notebooks = []
for id, row in tqdm(df_ancestors.iterrows()):
    if row['parent_id'] != None and cnt_by_group[row['ancestor_id']] == 1:
        good_notebooks.append(id)

good_notebooks = pd.Series(good_notebooks)

print('good notebooks', len(good_notebooks))

all_train_nb = good_notebooks.sample(frac=0.9, random_state=787788)
all_validate_nb = good_notebooks.drop(all_train_nb.index)

def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )

def load_train_nbs(num):
    global df

    paths_train = [data_dir / 'train' / '{}.json'.format(id) for id in all_train_nb.head(num)]
    notebooks_train = [
        read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
    ]
    df = (
        pd.concat(notebooks_train)
        .set_index('id', append=True)
        .swaplevel()
        .sort_index(level='id', sort_remaining=False)
    )

    df
    
def load_train_nbs_tail(num):
    global df

    paths_train = [data_dir / 'train' / '{}.json'.format(id) for id in all_train_nb.tail(num)]
    notebooks_train = [
        read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
    ]
    df = (
        pd.concat(notebooks_train)
        .set_index('id', append=True)
        .swaplevel()
        .sort_index(level='id', sort_remaining=False)
    )

    df
    
def load_validate_nbs(num=100000):
    global df

    paths_train = [data_dir / 'train' / '{}.json'.format(id) for id in all_validate_nb.head(num)]
    notebooks_train = [
        read_notebook(path) for path in tqdm(paths_train, desc='Validate NBs')
    ]
    df = (
        pd.concat(notebooks_train)
        .set_index('id', append=True)
        .swaplevel()
        .sort_index(level='id', sort_remaining=False)
    )

    df
    
load_train_nbs(100)

def get_example_nb_id(seed=0):
    return df.index.get_level_values(0).unique()[seed]
    
nb_id = get_example_nb_id()

def get_nb_by_id(nb_id):
    return df.loc[nb_id]

def get_example_cell_from_nb(nb):
    return nb.index[0]

def get_example_markdown_cell_from_nb(nb):
    return nb[nb['cell_type'] == 'markdown'].index[0]

def get_code_cells(nb):
    return nb[nb['cell_type'] == 'code'].index

def get_markdown_cells(nb):
    return nb[nb['cell_type'] == 'markdown'].index    


In [ ]:
# Copied from: https://github.com/microsoft/CodeBERT/blob/master/UniXcoder/unixcoder.py

# Copyright (c) Microsoft Corporation. 
# Licensed under the MIT license.

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig

class UniXcoder(nn.Module):
    def __init__(self, model_name, state_dict=None):
        """
            Build UniXcoder.
            Parameters:
            * `model_name`- huggingface model card name. e.g. microsoft/unixcoder-base
        """        
        super(UniXcoder, self).__init__()
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.config = RobertaConfig.from_pretrained(model_name)
        self.config.is_decoder = True
        self.model = RobertaModel.from_pretrained(model_name, config=self.config)

        if state_dict is not None:
            self.model.load_state_dict(torch.load(state_dict))   
        
        self.register_buffer("bias", torch.tril(torch.ones((1024, 1024), dtype=torch.uint8)).view(1,1024, 1024))
        self.lm_head = nn.Linear(self.config.hidden_size, self.config.vocab_size, bias=False)
        self.lm_head.weight = self.model.embeddings.word_embeddings.weight
        self.lsm = nn.LogSoftmax(dim=-1)
        
        self.tokenizer.add_tokens(["<mask0>"],special_tokens=True)
          
    def tokenize(self, inputs, mode="<encoder-only>", max_length=512, padding=False):
        """ 
        Convert string to token ids 
                
        Parameters:
        * `inputs`- list of input strings.
        * `max_length`- The maximum total source sequence length after tokenization.
        * `padding`- whether to pad source sequence length to max_length. 
        * `mode`- which mode the sequence will use. i.e. <encoder-only>, <decoder-only>, <encoder-decoder>
        """
        assert mode in ["<encoder-only>", "<decoder-only>", "<encoder-decoder>"]
        
        tokenizer = self.tokenizer
        
        tokens_ids = []
        for x in inputs:
            tokens = tokenizer.tokenize(x)
            if mode == "<encoder-only>":
                tokens = tokens[:max_length-4]
                tokens = [tokenizer.cls_token,mode,tokenizer.sep_token] + tokens + [tokenizer.sep_token]                
            elif mode == "<decoder-only>":
                tokens = tokens[-(max_length-3):]
                tokens = [tokenizer.cls_token,mode,tokenizer.sep_token] + tokens
            else:
                tokens = tokens[:max_length-5]
                tokens = [tokenizer.cls_token,mode,tokenizer.sep_token] + tokens + [tokenizer.sep_token]
                
            tokens_id = tokenizer.convert_tokens_to_ids(tokens)
            tokens_ids.append(tokens_id)

        if padding:
            cur_max_length = len(max(tokens_ids, key=len))
            tokens_ids = list(map(lambda l: l + [self.config.pad_token_id] * (cur_max_length-len(l)), tokens_ids))
        return tokens_ids
            
    def decode(self, source_ids):   
        """ Convert token ids to string """      
        predictions = []
        for x in source_ids:
            prediction = []
            for y in x:
                t = y.cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[:t.index(0)]
                text = self.tokenizer.decode(t,clean_up_tokenization_spaces=False)
                prediction.append(text)        
            predictions.append(prediction)
        return predictions
    
    def forward(self, source_ids):   
        """ Obtain token embeddings and sentence embeddings """
        mask = source_ids.ne(self.config.pad_token_id)
        token_embeddings = self.model(source_ids,attention_mask = mask.unsqueeze(1) * mask.unsqueeze(2))[0]
        sentence_embeddings = (token_embeddings * mask.unsqueeze(-1)).sum(1) / mask.sum(-1).unsqueeze(-1)
        return token_embeddings, sentence_embeddings       

    def generate(self, source_ids, decoder_only = True, eos_id = None, beam_size = 5, max_length = 64):
        """ Generate sequence given context (source_ids) """
        
        # Set encoder mask attention matrix: bidirectional for <encoder-decoder>, unirectional for <decoder-only>
        if decoder_only:
            mask = self.bias[:,:source_ids.size(-1),:source_ids.size(-1)]
        else:
            mask = source_ids.ne(self.config.pad_token_id)
            mask = mask.unsqueeze(1) * mask.unsqueeze(2)  
            
        if eos_id is None:
            eos_id = self.config.eos_token_id
        
        device = source_ids.device
        
        # Decoding using beam search
        preds = []       
        zero = torch.LongTensor(1).fill_(0).to(device)   
        source_len = list(source_ids.ne(1).sum(-1).cpu().numpy())
        length = source_ids.size(-1)
        encoder_output = self.model(source_ids,attention_mask=mask)
        for i in range(source_ids.shape[0]):
            context = [[x[i:i+1,:,:source_len[i]].repeat(beam_size,1,1,1) for x in y] 
                     for y in encoder_output.past_key_values]
            beam = Beam(beam_size,eos_id,device)
            input_ids = beam.getCurrentState().clone()
            context_ids = source_ids[i:i+1,:source_len[i]].repeat(beam_size,1)
            out = encoder_output.last_hidden_state[i:i+1,:source_len[i]].repeat(beam_size,1,1)
            for _ in range(max_length): 
                if beam.done():
                    break
                if _ == 0: 
                    hidden_states = out[:,-1,:]
                    out = self.lsm(self.lm_head(hidden_states)).data
                    beam.advance(out)
                    input_ids.data.copy_(input_ids.data.index_select(0, beam.getCurrentOrigin()))
                    input_ids = beam.getCurrentState().clone()
                else:
                    length = context_ids.size(-1)+input_ids.size(-1)
                    out = self.model(input_ids,attention_mask=self.bias[:,context_ids.size(-1):length,:length],
                                       past_key_values=context).last_hidden_state
                    hidden_states = out[:,-1,:]
                    out = self.lsm(self.lm_head(hidden_states)).data
                    beam.advance(out)
                    input_ids.data.copy_(input_ids.data.index_select(0, beam.getCurrentOrigin()))
                    input_ids = torch.cat((input_ids,beam.getCurrentState().clone()),-1)
            hyp = beam.getHyp(beam.getFinal())
            pred = beam.buildTargetTokens(hyp)[:beam_size]
            pred = [torch.cat([x.view(-1) for x in p]+[zero]*(max_length-len(p))).view(1,-1) for p in pred]
            preds.append(torch.cat(pred,0).unsqueeze(0))

        preds = torch.cat(preds,0)    

        return preds  
    

    
class Beam(object):
    def __init__(self, size, eos, device):
        self.size = size
        self.device = device
        # The score for each translation on the beam.
        self.scores = torch.FloatTensor(size).zero_().to(device)
        # The backpointers at each time-step.
        self.prevKs = []
        # The outputs at each time-step.
        self.nextYs = [torch.LongTensor(size).fill_(0).to(device)]
        # Has EOS topped the beam yet.
        self._eos = eos
        self.eosTop = False
        # Time and k pair for finished.
        self.finished = []

    def getCurrentState(self):
        "Get the outputs for the current timestep."
        batch = self.nextYs[-1].view(-1, 1)
        return batch

    def getCurrentOrigin(self):
        "Get the backpointers for the current timestep."
        return self.prevKs[-1]

    def advance(self, wordLk):
        """
        Given prob over words for every last beam `wordLk` and attention
        `attnOut`: Compute and update the beam search.
        Parameters:
        * `wordLk`- probs of advancing from the last step (K x words)
        * `attnOut`- attention at the last step
        Returns: True if beam search is complete.
        """
        numWords = wordLk.size(1)

        # Sum the previous scores.
        if len(self.prevKs) > 0:
            beamLk = wordLk + self.scores.unsqueeze(1).expand_as(wordLk)

            # Don't let EOS have children.
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] == self._eos:
                    beamLk[i] = -1e20
        else:
            beamLk = wordLk[0]
        flatBeamLk = beamLk.view(-1)
        bestScores, bestScoresId = flatBeamLk.topk(self.size, 0, True, True)

        self.scores = bestScores

        # bestScoresId is flattened beam x word array, so calculate which
        # word and beam each score came from
        prevK = bestScoresId // numWords
        self.prevKs.append(prevK)
        self.nextYs.append((bestScoresId - prevK * numWords))


        for i in range(self.nextYs[-1].size(0)):
            if self.nextYs[-1][i] == self._eos:
                s = self.scores[i]
                self.finished.append((s, len(self.nextYs) - 1, i))

        # End condition is when top-of-beam is EOS and no global score.
        if self.nextYs[-1][0] == self._eos:
            self.eosTop = True

    def done(self):
        return self.eosTop and len(self.finished) >= self.size

    def getFinal(self):
        if len(self.finished) == 0:
            self.finished.append((self.scores[0], len(self.nextYs) - 1, 0))
        self.finished.sort(key=lambda a: -a[0])
        if len(self.finished) != self.size:
            unfinished=[]
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] != self._eos:
                    s = self.scores[i]
                    unfinished.append((s, len(self.nextYs) - 1, i)) 
            unfinished.sort(key=lambda a: -a[0])
            self.finished+=unfinished[:self.size-len(self.finished)]
        return self.finished[:self.size]

    def getHyp(self, beam_res):
        """
        Walk back to construct the full hypothesis.
        """
        hyps=[]
        for _,timestep, k in beam_res:
            hyp = []
            for j in range(len(self.prevKs[:timestep]) - 1, -1, -1):
                hyp.append(self.nextYs[j+1][k])
                k = self.prevKs[j][k]
            hyps.append(hyp[::-1])
        return hyps
    
    def buildTargetTokens(self, preds):
        sentence=[]
        for pred in preds:
            tokens = []
            for tok in pred:
                if tok==self._eos:
                    break
                tokens.append(tok)
            sentence.append(tokens)
        return sentence
        

In [ ]:
import torch

def reload_model(preload_from_working=False, preload_from_test=None):
    global unixcoder_model
    global device
    default_model_name = "../input/unixcoderbase"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    state_dict = None
    if preload_from_test is not None:
        state_dict = "../input/ai4code-train-ds/" + preload_from_test
        print("Preloading from test..", state_dict)
    unixcoder_model = UniXcoder(model_name=default_model_name, state_dict=state_dict)
    unixcoder_model.to(device)

reload_model()

df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list

In [ ]:
@torch.no_grad()
def get_embedding(nb, cell_id):
    start = time.time()
    cell = nb.loc[cell_id]
    tokens = unixcoder_model.tokenize([cell['source']],max_length=512,mode="<encoder-only>")
    source_ids = torch.tensor(tokens).to(device)
    _,embeddings = unixcoder_model(source_ids)
    return torch.nn.functional.normalize(embeddings, p=2, dim=1)[0].cpu()

In [ ]:
def get_text_tokens(text):
    tokens = unixcoder_model.tokenize([text],max_length=512,mode="<encoder-only>")
    return torch.tensor(tokens).to(device)

def get_texts_tokens(texts):
    tokens = unixcoder_model.tokenize(texts,max_length=512,mode="<encoder-only>", padding=True)
    return torch.tensor(tokens).to(device)
    

def get_text_embedding(text):
    source_ids = get_text_tokens(text)
    _,embeddings = unixcoder_model(source_ids)
    return torch.nn.functional.normalize(embeddings, p=2, dim=1).cpu()[0]

In [ ]:
run_preproc = False

@torch.no_grad()
def get_nb_embeddings(nb):
    start = time.time()

    res = {}
    
    code_cells = nb[nb['cell_type'] == 'code'].reset_index(level='cell_id')
    code_cell_ids = code_cells['cell_id'].values.tolist()
        

#     TODO: maybe different size?
    batch_size = 10
    n_chunks = len(nb) / min(len(nb), batch_size)

    nb = nb.sort_values(by="source", key=lambda x: x.str.len())
    for nb in np.array_split(nb, n_chunks):
        # TODO: different max_length?
        
        texts = []
        for key,text in zip(nb['source'].index, nb['source'].to_numpy()):
            if key in code_cell_ids or not run_preproc:
                texts.append(text)
            else:
                texts.append(text.lower())
        
        tokens = unixcoder_model.tokenize(texts,max_length=512,mode="<encoder-only>", padding=True)
        source_ids = torch.tensor(tokens).to(device)
        _,embeddings = unixcoder_model(source_ids)
        normalized = torch.nn.functional.normalize(embeddings, p=2, dim=1).cpu()

        
        for key, val in zip(nb['source'].index, normalized):
            res[key] = val
            
    res['END'] = get_text_embedding('END')
    
    return res

def get_source(cell_id):
    return nb.loc[cell_id]['source']

def sim(emb1, emb2):
    return torch.einsum("i,i->", emb1, emb2).detach().numpy()

In [ ]:
from bisect import bisect


# Actually O(N^2), but fast in practice for our data
def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):  # O(N)
        j = bisect(sorted_so_far, u)  # O(log N)
        inversions += i - j
        sorted_so_far.insert(j, u)  # O(N)
    return inversions

def kendall_tau(ground_truth, predictions):
    total_inversions = 0  # total inversions in predicted ranks across all instances
    total_2max = 0  # maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return [1 - 4 * total_inversions / total_2max, total_inversions, total_2max]

def sum_scores(a, b):
    total_inversions = a[1] + b[1]
    total_2max = a[2] + b[2]
    return [1 - 4 * total_inversions / total_2max, total_inversions, total_2max]

In [ ]:
paths_test = list((data_dir / 'test').glob('*.json'))
notebooks_test = [
    read_notebook(path) for path in tqdm(paths_test, desc='Test NBs')
]
test_df = (
    pd.concat(notebooks_test)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)

In [ ]:
import math 

@torch.no_grad()
def find_best_cell_order3_with_emb(nb, embeddings):
    coef_mul = 1000.0
    
    code_cells = nb[nb['cell_type'] == 'code'].reset_index(level='cell_id')
    markdown_cells = nb[nb['cell_type'] != 'code'].reset_index(level='cell_id')
        
    code_cell_ids = code_cells['cell_id'].values.tolist()
    code_cell_ids.append('END')
    
    order = code_cell_ids.copy()    
    
    for m_cell_id in markdown_cells['cell_id'].values:
        markdown_emb = embeddings[m_cell_id]
        sims = [sim(markdown_emb, embeddings[c]) for c in code_cell_ids]
        max_sim = max(sims)
        sims_probs = list(map(lambda x:math.exp((x-max_sim) * coef_mul), sims))
        sum_probs = sum(sims_probs)
        sims_probs = list(map(lambda x:x/sum_probs, sims_probs))
        scores = [0.0] * len(sims_probs)
        for i in range(len(sims)):
            for j in range(len(sims)):
                scores[j] += abs(i - j) * sims_probs[i]
        best_pos = scores.index(min(scores))
        
        order.insert(order.index(code_cell_ids[best_pos]), m_cell_id)
    
    assert order[-1] == 'END'
    order.pop()
    return order

@torch.no_grad()
def find_best_cell_order3(nb):
    embeddings = get_nb_embeddings(nb)
    return find_best_cell_order3_with_emb(nb, embeddings)

In [ ]:
def save_results():
    res = []

    for id_, notebook in test_df.groupby(level="id"):
        order = find_best_cell_order3(notebook.reset_index(level='id'))
        order = " ".join(order)
        
        res.append({'id' : id_, 'cell_order' : order})
        
    
    res = pd.DataFrame(res)
    res.to_csv('submission.csv', index=False)

    display(res.head())

In [ ]:
reload_model(preload_from_test=default_model_name)

In [ ]:
save_results()